In [ ]:
!pip install daft deltalake<0.17

In [ ]:
CI = False

In [ ]:
# Skip this notebook execution in CI because it requires AWS credentials for presigned URL generation
if CI:
    import sys

    sys.exit()

## Multimodal data lake annotation and indexing

Let's go from: **Images in an S3 Bucket**

To: **Multimodal Data Lake** where we can run queries efficiently to power analytics, retrieval and more!

In [1]:
import daft

IO_CONFIG = daft.io.IOConfig(s3=daft.io.S3Config(anonymous=True))  # Use anonymous S3 access

daft.set_planning_config(default_io_config=IO_CONFIG)

DaftContext(_daft_execution_config=<daft.daft.PyDaftExecutionConfig object at 0x103a12b90>, _daft_planning_config=<daft.daft.PyDaftPlanningConfig object at 0x1037aff90>, _runner_config=_PyRunnerConfig(use_thread_pool=None), _disallow_set_runner=False, _runner=None)

In [2]:
df = daft.from_glob_path(
    "s3://daft-public-data/open-images/validation-images/*",
)
df.show()

pathUtf8,sizeInt64,num_rowsInt64
s3://daft-public-data/open-images/validation-images/0001eeaf4aed83f9.jpg,290621,None
s3://daft-public-data/open-images/validation-images/0004886b7d043cfd.jpg,375363,None
s3://daft-public-data/open-images/validation-images/000595fe6fee6369.jpg,462817,None
s3://daft-public-data/open-images/validation-images/00075905539074f2.jpg,302326,None
s3://daft-public-data/open-images/validation-images/0007cebe1b2ba653.jpg,970275,None
s3://daft-public-data/open-images/validation-images/0007d6cf88afaa4a.jpg,614095,None
s3://daft-public-data/open-images/validation-images/0008e425fb49a2bf.jpg,415082,None
s3://daft-public-data/open-images/validation-images/0009bad4d8539bb4.jpg,359851,None


### Working with URLs in Daft is really **easy and efficient**

* URLs are extremely common when working with multimodal data, most commonly as a `https://` URL or `s3://` object store URL
* Daft runs URL downloads using async Rust kernels, saturating your machine's network bandwidth even for millions of small files (see: [demo at PyData Global 2023](https://www.youtube.com/watch?v=QEOFwptwnXQ&ab_channel=PyData))

In [3]:
df = df.with_column("image_bytes", df["path"].url.download())
df.show()

pathUtf8,sizeInt64,num_rowsInt64,image_bytesBinary
s3://daft-public-data/open-images/validation-images/0001eeaf4aed83f9.jpg,290621,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/0004886b7d043cfd.jpg,375363,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/000595fe6fee6369.jpg,462817,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/00075905539074f2.jpg,302326,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/0007cebe1b2ba653.jpg,970275,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/0007d6cf88afaa4a.jpg,614095,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/0008e425fb49a2bf.jpg,415082,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."
s3://daft-public-data/open-images/validation-images/0009bad4d8539bb4.jpg,359851,None,"b""\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01""..."


### Reading Images

Daft makes working with opaque file formats/encodings easy

* Native type available for images and tensors
* Support for arbitrary Python objects in columns so you can use all your favorite Python libraries as well for datatypes not yet supported by Daft (e.g. video, audio, PDFs)

In [4]:
df = df.with_column("image", df["image_bytes"].image.decode())
df.show()

╭────────────────────────────────┬────────┬──────────┬────────────────────────────────┬──────────────╮
│ path                           ┆ size   ┆ num_rows ┆ image_bytes                    ┆ image        │
│ ---                            ┆ ---    ┆ ---      ┆ ---                            ┆ ---          │
│ Utf8                           ┆ Int64  ┆ Int64    ┆ Binary                         ┆ Image[MIXED] │
╞════════════════════════════════╪════════╪══════════╪════════════════════════════════╪══════════════╡
│ s3://daft-public-data/open-im… ┆ 290621 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 375363 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 462817 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 302326 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 970275 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 614095 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 415082 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 359851 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      │
╰────────────────────────────────┴────────┴──────────┴────────────────────────────────┴──────────────╯

(Showing first 8 rows)

### Thumbnail creation

Easily create thumbnails for your image using the `.image.resize(...)` Daft expression.

In [5]:
df = df.with_column("image_thumbnail", df["image"].image.resize(32, 32))
df.show()

╭────────────────────────────────┬────────┬──────────┬────────────────────────────────┬──────────────┬─────────────────╮
│ path                           ┆ size   ┆ num_rows ┆ image_bytes                    ┆ image        ┆ image_thumbnail │
│ ---                            ┆ ---    ┆ ---      ┆ ---                            ┆ ---          ┆ ---             │
│ Utf8                           ┆ Int64  ┆ Int64    ┆ Binary                         ┆ Image[MIXED] ┆ Image[MIXED]    │
╞════════════════════════════════╪════════╪══════════╪════════════════════════════════╪══════════════╪═════════════════╡
│ s3://daft-public-data/open-im… ┆ 290621 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 375363 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 462817 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 302326 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 970275 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 614095 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 415082 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-data/open-im… ┆ 359851 ┆ None     ┆ b"\xff\xd8\xff\xe0\x00\x10JFI… ┆ <Image>      ┆ <Image>         │
╰────────────────────────────────┴────────┴──────────┴────────────────────────────────┴──────────────┴─────────────────╯

(Showing first 8 rows)

### Running multimodal LLMs

Since we are running on just our laptop, we will be offloading our "heavy compute" (running the GPT-4o model on our image) to the OpenAI API.

If instead we wanted to run our own models or algorithms, Daft also lets us run on GPUs with the `df.with_column(..., resource_request=ResourceRequest(num_gpus=1))` pattern.

In [6]:
import json
import os

import boto3
import requests

DEFAULT_PROMPT = "What’s in this image?"
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise RuntimeError("Please specify your OpenAI API key as the environment variable `OPENAI_API_KEY`.")

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}


@daft.udf(return_dtype=daft.DataType.string())
def generate_presigned_url(s3_urls, expires_in=3600):
    """Generate a presigned Amazon S3 URLs."""
    s3_client = boto3.client("s3")
    presigned_urls = []
    for s3_url in s3_urls.to_pylist():
        bucket, key = s3_url.strip("s3://").split("/", 1)
        url = s3_client.generate_presigned_url(
            ClientMethod="get_object", Params={"Bucket": bucket, "Key": key}, ExpiresIn=expires_in
        )
        presigned_urls.append(url)
    return presigned_urls


@daft.udf(return_dtype=daft.DataType.string())
def run_gpt4o_on_urls(images_urls, prompt=DEFAULT_PROMPT):
    """Run the gpt-4o LLM by making an API call to OpenAI."""
    results = []
    for url in images_urls.to_pylist():
        payload = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What’s in this image?"},
                        {"type": "image_url", "image_url": {"url": url}},
                    ],
                }
            ],
            "max_tokens": 300,
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        results.append(json.dumps(response.json()))

    return results

In [7]:
# Generate temporary URLs with a short expiration time
df = df.with_column("image_urls", generate_presigned_url(df["path"]))

# Make remote API calls to OpenAI endpoint
df = df.with_column("gpt_results", run_gpt4o_on_urls(df["image_urls"], prompt="What’s in this image?"))

# Parse JSON outputs from OpenAI endpoint
df = df.with_column("description", df["gpt_results"].json.query(".choices[0].message.content"))

df.show(3)

╭─────────────────────┬────────┬──────────┬────────────┬─────────────────────┬────────────────────┬────────────────────╮
│ path                ┆ size   ┆ num_rows ┆      …     ┆ image_urls          ┆ gpt_results        ┆ description        │
│ ---                 ┆ ---    ┆ ---      ┆            ┆ ---                 ┆ ---                ┆ ---                │
│ Utf8                ┆ Int64  ┆ Int64    ┆ (3 hidden) ┆ Utf8                ┆ Utf8               ┆ Utf8               │
╞═════════════════════╪════════╪══════════╪════════════╪═════════════════════╪════════════════════╪════════════════════╡
│ s3://daft-public-da ┆ 290621 ┆ None     ┆ …          ┆ https://daft-public ┆ {"id": "chatcmpl-9 ┆ "The image shows   │
│ ta/open-im…         ┆        ┆          ┆            ┆ -data.s3.a…         ┆ YyMF9piO5bA…       ┆ the apron ar…      │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-da ┆ 375363 ┆ None     ┆ …          ┆ https://daft-public ┆ {"id": "chatcmpl-9 ┆ "The image depicts │
│ ta/open-im…         ┆        ┆          ┆            ┆ -data.s3.a…         ┆ YyMKH8QdViB…       ┆ a collecti…        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ s3://daft-public-da ┆ 462817 ┆ None     ┆ …          ┆ https://daft-public ┆ {"id": "chatcmpl-9 ┆ "The image shows a │
│ ta/open-im…         ┆        ┆          ┆            ┆ -data.s3.a…         ┆ YyMQg1KK0zO…       ┆ branch of …        │
╰─────────────────────┴────────┴──────────┴────────────┴─────────────────────┴────────────────────┴────────────────────╯

(Showing first 3 rows)

In [9]:
df = df.select(
    # Larger multimodal data (such as large images or documents) can be written as URLs
    "path",
    # Small multimodal data (such as thumbnails or full-form text) can be written inline
    df["image_thumbnail"].image.encode("JPEG"),
    # Metadata such as size in bytes and descriptions should be stored as per normal
    "size",
    "description",
)

DaftCoreException: DaftError::External Unable to create logical plan node.
Due to: DaftError::TypeError ImageEncode can only encode ImageArrays and FixedShapeImageArrays, got image_thumbnail#Binary

In [10]:
df

pathUtf8,image_thumbnailBinary,sizeInt64,descriptionUtf8


In [11]:
# Limit to running just 8 rows to save your OpenAI bill...
df = df.limit(8)

df.write_delta("my_table.delta_lake")

## Now we have our "Multimodal Data Lake"!

1. Thumbnails readily available for visualization
2. URLs available for access to the raw data
3. Extracted metadata (`description`) available for querying

In [12]:
read_df = daft.read_deltalake("my_table.delta_lake")
read_df

pathUtf8,image_thumbnailBinary,sizeInt64,descriptionUtf8


In [13]:
read_df = read_df.with_column("image_thumbnail", daft.col("image_thumbnail").image.decode()).where(
    read_df["description"].str.contains("dog")
)

In [14]:
read_df.collect()

pathUtf8,sizeInt64,descriptionUtf8,image_thumbnailImage[MIXED]
s3://daft-public-data/open-images/validation-images/0007cebe1b2ba653.jpg,970275,"""The image depicts a black and white dog, likely a Border Collie, energetically running on a grassy area in a park. In the background, a person wearing pink pants is visible, suggesting they might be walking or playing with the dog. Some trees and a large stick on the ground are also visible in the scene.""","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAgAAAQABAAD/wAARCACAAIADAREAAhEBAxEB/9sAQwAIBgYHBgUIBwcHCQkICgwUDQwLCwwZEhMPFB0aHx4dGhwcICQuJyAiLCMcHCg3KSwwMTQ0NB8nOT04MjwuMzQy/9sAQwEJCQkMCwwYDQ0YMiEcITIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDFu4T5Z+lfOUdz3q+ITRgROYrjp/FXpp6HkTldnZ6Xe4ReO1c9SRvTNltU8sA7ahSOgnttfycbK0VVENEl9ficLxVe1RlJD9MlUbssBUuoZuJomSMj76/nWTkTYgZ0b+IVk5AQtGjA/OPzpXEzG1C0XY53jpWsRHMWyiOdxnqa6IoVjQP7qSNvetqcjro4Zs6y11NJbQpkD5cVdR6GrwjsznLzT1mMjbxznvXmTep58YcjZz39n+VI3Oea3pTsKD1NO7kAjbjtXPSRDrNnLk77jgfxV1t6CizqNNG1Bn0riqT1OqE7GjKN2KhSNPbIfHCEwc0uZke2LDjeBg1Skw9oRgSR5wTVakuZGZZRkl2xTs2ZuoR/2iq5BmGfrVezYcwJqJf7r5/Go5bBchvrmXyz8rHIq4uwznommMxOxuvpWymXHc1iWkjBweBWtM+jwcE0V4r6SJiMmuhq6PRjQjKLHvqcijoea4KsNT5PGw5JMYt1u5K1iro4aaHagR5Zwe1VTMImNYwCS4PPet5bGmx1cUAjjXntXDNXZlKoTgg9TVwgLnbHEf7VX7ISbJbdhzk1apm8bj57hIx2rRUxyTMy91BVt5MAfdrWNLUwd7nml5rEq3MmHbG71rqdJWNkzX8Pa0zzAPnr3rkq07D5j0cPFLbodq8iuOSsPmK/lRLn92vPtUxvcuMtUQPGqo/SvVhGyPocLPliYLYWfn1rbQ9LBV+dSRoNCkir0rKcLnz+Zr3hX03IBBrmdI8yDsULs7lPzVzQOKEilYjZMee9bPY0k9DojN+7HPaudx1OfqZ8968Z4BrppxR004pkaau+DlD+VaWRtGCFj1t1z8ho0OuFOJWutbkYj5TWkbFSpxKdzqDyQMMHkVtFo5p04nLQ2Bup5CTjmtXLQ5ma9hpq2sqnzAOfWuWpqI7aK72RIAc4Fc7gK5LJqHAAWkqauVTu5Ije4LLx3rseiPqORxpX8jJvAd4PvWftDHK6vvSTJIpWG3knFap3MMzTvc0RqOxQNual2PElLsc9dTkCuPlIjTRBazkP+NNo0cEbcUxKis2jCUEOyG6rWd2CVuohhU9hVczKUmRm1U+lPmZam+5G1kvoKOZj533K09qBE/HaqjJ3I52UtC0z7Y05zjbmvTUdC0UNWJtZgFfOD2NS4FMv6ZrJCYcdu9Yypsg1bfUFlJHHNZKm7nVQtzo0ogUQsRwRXVW+E+mxFSPsl6GfcSh2IxXmpu587h6lqugWyfeJrti9D2sdJOkhzSAGuSs3c+UpfEzn57zcPu1vYqNNkUNyQ33allumzYt73gcVlI5pQZejn3D7tYslQY8uaCgEh9KLljwxPai4mxskfmIVx1FOL95E3Lvh7R/Jtrts8spr2oNcqNonGz6JPd6i4IbG+tNCmXtT8NSabbI/PK5o5UZ3MKwvjFeBCf4sVDgjopSs0ehiaOSzXBGdtc9d6HoV8TeFjDkBEh+ted1PKoz9+5ZtzlTXVF6HsYitzQsQshLGuepqeJT0kzEFuG71tzFRqMkW1UdxQ2aObLEMagj5hWbMJSZqwbAv3hWTEmx4Kk9RSsVyliOJW7ikPlLIgXHUVImhrRhaFuZ21LFlqoswyMnDcV6kJ6HRFGto9pDd3Ik2L97NbxkNon+IKQW+nxgBc7BWqZieDMc3zFT/AB0N6G0Ts9FlkmjCnOMV51WZM5tmu9lkZrjb1MoKzuMgh2bq0UzrlUurCEAGpk7nMtGYgtCO9PmKjUQG2b1NVc29ohotnHc0NmbmiQLIndqkpSRMJpB/CaWg9CaK8kX+E0mGheivS3WoZLSLSSF8cUluZ2K+pBiUKqRXoR2Nom/4buzbxOzdhW8Rs5jx/wCIzd7Y1PTit0YM85tkdrtSATualJqxotj0vSbUQ26tjqK8mctTLqaXmAgjFc7ZpZEOBzQmQmVSmSaq4SKApmRIqg0DuTLCDS1AkFup7CkA/wCzovYUtQuxwt1YcAU7juyI2eGyDQw1NC0i244zSUXcZPfbdg+QcCvQeyNblW3u/LgmHTg06b1C5werRtc3DEk/eruUlYRPomm/vMlc4NefXqCO1jXbGo9BXA2QBFAXI2FMCMp70wMkGtLEkqyYpWAmWbFTYCVbiiwD2mzihoCeGTisWANOCwGKuJRr6fEsq5rphEYmoxLGByK6JbFnP3OVO1e/pUJ2EZF3p7K6tg8mrdTQRsaXbLGnIGcVwVZ3Yrl8jFZiGGgCNjTAiLUwMmumwrCilYLDxSsFh4FKwDy2MUNATQzdqycRj2PIOaqKsMu22qC3GK6IsZDeap9p6GruAabb/adzMfu881LFcfdvGQRhflrOTEU7W5yzAdq5pRuK5a+0ClYYhnBqrANLg07ANyKdgMUNXQA8NQA8PSsA4PSAcDupALnZ3quUBTKSKViirKzHoTU3Ar+Y8TDqc1qmI17DUjBEw2/eFUxFVpWkdjk8msmSOjHk985pWQEm4msrDHBjTsMXf707AG+iwGUvNdJRIKQDhSAkFIAzQKw4ZoAfSGNK5pDEMQPagQojxSELspMQu2kIUHFIQpanYBhNUAoamBRWtTUlApAOApASAcUhjQOaYWJAtIQlICQDNIBdtAgxQIXFKwgxRYBClKw7DMUwsLso